In [21]:
import numpy as np
class KeyClustering():
  def __init__(self, data, C_target):
    self.C_target = C_target
    self.N = len(data)
    self.k = 1
    self.g = 2

    self.D_orginal = np.zeros((self.N, self.N))
    self.R = np.zeros((self.N, self.k+1), dtype=np.uint)
    self.L = np.array([i for i in range(self.N)])
    self.D_current = np.zeros((self.N, self.N))

    self.init_C()

    self.init_D_orginal(data)

    self.init_R()

    self.init_D_current()

    
  def fit(self):
    while self.C_current > self.C_target:
      self.S_current, self.K_current = self.find_key_clusters(self.C_current)
      
      self.update_labels()
      
      self.D_current = self.update_new_distances()
      
      self.update_C()

    
    self.C_current = self.C_target
    self.S_current, self.K_current = self.find_key_clusters(self.C_current)

    self.update_labels()
    return self.L
    

  def init_C(self):
    self.C_previous = self.N
    self.C_current = self.N // self.g

  def dist(self, a, b):
    return np.abs(a - b).sum()

  def init_D_orginal(self, data):
    for i in range(self.N):
      for j in range(self.N):
        self.D_orginal[i, j] = self.dist(data[i], data[j])

  def init_R(self):
    D_copy = self.D_orginal.copy()
    max_val = self.D_orginal.max() + 1
    for i in range(self.N):
      self.R[i, 0] = i
      D_copy[i, i] = max_val
      for j in range(1, self.k+1):
        self.R[i, j] = np.argmin(D_copy[i])
        D_copy[i, self.R[i, j]] = max_val

  def init_D_current(self):
    const = 1 / (self.k+1)**2
    for i in range(self.N):
      for j in range(self.N):
        sum = 0
        for a in self.R[i]:
          for b in self.R[j]:
            if i != j:
              sum += self.D_orginal[a, b]
        self.D_current[i, j] = const * sum

  def find_key_clusters(self, C_stop):
    distance = np.array([np.sum(self.D_current[i]) for i in range(self.C_current)])

    I1 = np.argmin(distance)

    K_curr = []
    for i in range(len(self.D_current)):
      if i != I1:
        K_curr.append(i)
    K_curr = np.array(K_curr)
    
    S_curr = [I1]
    
    n = 1
    while n != C_stop:
      
      next_key_distances = []
      for item_k in K_curr:
        tmp = []
        for item_S in S_curr:
          tmp.append(self.D_current[item_k, item_S])
          
        next_key_distances.append(np.min(tmp))
      
      I_n = np.argmax(next_key_distances)
      
      S_curr.append(K_curr[I_n])
      K_curr = np.delete(K_curr, I_n)

 
      n += 1
    return np.array(S_curr), K_curr

  def update_labels(self):
    for item_K in self.K_current:
      tmp = []
      for item_S in self.S_current:
        tmp.append(self.D_current[item_K, item_S])

      arg_min = np.argmin(tmp)
      
      self.L[np.where(self.L==item_K)] = self.S_current[arg_min]


    new_label = {}
    for i in range(self.C_current):
      new_label[self.S_current[i]] = i

    for i in range(len(self.L)):
      self.L[i] = new_label[self.L[i]]

  def update_new_distances(self):
    D_curr = np.zeros((self.C_current, self.C_current))

    clusters = []
    for i in range(self.C_current):
      clusters.append([])

    for index, item in enumerate(self.L):
      clusters[item].append(index)

    for i in range(self.C_current):
      c_1_len = len(clusters[i])
      for j in range(self.C_current):
        if i == j:
          continue
        c_2_len = len(clusters[j])

        sum = 0
        for c1 in clusters[i]:
          for c2 in clusters[j]:
            sum += self.D_orginal[c1, c2]
        D_curr[i, j] = 1 / (c_1_len * c_2_len) * sum
    return D_curr

  def update_C(self):
    self.C_previous = self.C_current
    self.C_current = self.C_current // self.g

d = np.array([0xff0000, 0xff0001, 0xff0002, 0xff0003, 0x00ff01, 0x00ff02, 0x0000fd, 0x0000fe, 0x0000ff])
np.random.shuffle(d)
for item in d:
  print(hex(item), end= ' ')
print()
KC = KeyClustering(d, 3)
labels = KC.fit()
labels

0xff0001 0xfd 0xff0000 0xff0003 0xff01 0xff 0xfe 0xff02 0xff0002 


array([1, 2, 1, 1, 0, 2, 2, 0, 1])